<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/openairinterface/oai_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('/content/sdr-test-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.7/487.7 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import packages

In [ ]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr

In the next cell, we reserve the required equipment and resources to form an end to end sdr 5g setup. We reserve 2 SDRs and 3 workers.

In [ ]:
# reserve worker-01
worker01_lease = show_reservation_byname("worker-01-lease")
if not worker01_lease:
    worker01_lease = reserve(
        { "type":"device", "name":"worker-01", "duration": { "days":7, "hours":0 } }
    )
worker01_reservation_id = worker01_lease["reservations"][0]["id"]

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

In the following section we setup the 5g core (refer to [here](https://github.com/KTH-EXPECA/oai5g-docker/blob/main/docs/5g-sa-chi-tutorial.md) for more info).
It creates:

1.   5G core network with subnet "192.168.70.128/26"
2.   MySQL, UDR, UDM, AUSF, AMF, SMF, and SPGWU services on worker-01, on interface ens1f1.

In [ ]:
# create or get 5g core's network
corenet = None
try:
    corenet = chi.network.get_network("5gcn-net")
except Exception as ex:
    logger.info(f"could not find 5gcn-net, creating it...")

if not corenet:
    corenet = chi.network.create_network("5gcn-net")
    chi.network.create_subnet("5gcn-net-subnet",corenet["id"],"192.168.70.128/26",enable_dhcp=False)
    logger.success("5gcn-net is created.")

# run MySQL
status = get_container_status("5gcn-1-mysql")
if not status:
    chi.container.create_container(
        name = "5gcn-1-mysql",
        image = "samiemostafavi/expeca-mysql",
        reservation_id = worker01_reservation_id,
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.131/26"},
    )
    chi.container.wait_for_active("5gcn-1-mysql")
logger.success("5gcn-1-mysql is up.")

# run UDR
status = get_container_status("5gcn-2-udr")
if not status:
    chi.container.create_container(
        name = "5gcn-2-udr",
        image = "samiemostafavi/expeca-udr",
        reservation_id = worker01_reservation_id,
        environment = {"UDR_INTERFACE_NAME_FOR_NUDR":"net1","USE_FQDN_DNS":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.136/26"},
    )
    chi.container.wait_for_active("5gcn-2-udr")
logger.success("5gcn-2-udr is up.")

# run UDM
status = get_container_status("5gcn-3-udm")
if not status:
    chi.container.create_container(
        name = "5gcn-3-udm",
        image = "samiemostafavi/expeca-udm",
        reservation_id = worker01_reservation_id,
        environment = {"SBI_IF_NAME":"net1","USE_FQDN_DNS":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.137/26"},
    )
    chi.container.wait_for_active("5gcn-3-udm")
logger.success("5gcn-3-udm is up.")

# run AUSF
status = get_container_status("5gcn-4-ausf")
if not status:
    chi.container.create_container(
        name = "5gcn-4-ausf",
        image = "samiemostafavi/expeca-ausf",
        reservation_id = worker01_reservation_id,
        environment = {"SBI_IF_NAME":"net1","USE_FQDN_DNS":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.138/26"},
    )
    chi.container.wait_for_active("5gcn-4-ausf")
logger.success("5gcn-4-ausf is up.")

# run AMF
status = get_container_status("5gcn-5-amf")
if not status:
    chi.container.create_container(
        name = "5gcn-5-amf",
        image = "samiemostafavi/expeca-amf",
        reservation_id = worker01_reservation_id,
        environment = {"AMF_INTERFACE_NAME_FOR_NGAP":"net1","AMF_INTERFACE_NAME_FOR_N11":"net1","USE_FQDN_DNS":"no","NF_REGISTRATION":"no","SMF_SELECTION":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.132/26"},
    )
    chi.container.wait_for_active("5gcn-5-amf")
logger.success("5gcn-5-amf is up.")

# run SMF
status = get_container_status("5gcn-6-smf")
if not status:
    chi.container.create_container(
        name = "5gcn-6-smf",
        image = "samiemostafavi/expeca-smf",
        reservation_id = worker01_reservation_id,
        environment = {"USE_FQDN_DNS":"no","SMF_INTERFACE_NAME_FOR_N4":"net1","SMF_INTERFACE_NAME_FOR_SBI":"net1","REGISTER_NRF":"no","DISCOVER_UPF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {"networks.1.interface":"ens1f1","networks.1.ip":"192.168.70.133/26"},
    )
    chi.container.wait_for_active("5gcn-6-smf")
logger.success("5gcn-6-smf is up.")


# run SPGWU
status = get_container_status("5gcn-7-spgwu")
if not status:
    chi.container.create_container(
        name = "5gcn-7-spgwu",
        image = "samiemostafavi/expeca-spgwu",
        reservation_id = worker01_reservation_id,
        environment = {"SGW_INTERFACE_NAME_FOR_S1U_S12_S4_UP":"net1","SGW_INTERFACE_NAME_FOR_SX":"net1","PGW_INTERFACE_NAME_FOR_SGI":"net1","USE_FQDN_NRF":"no","REGISTER_NRF":"no"},
        nets = [{ "network" : corenet['id'] }],
        labels = {
            "networks.1.interface":"ens1f1",
            "networks.1.ip":"192.168.70.134/26",
            "capabilities.privileged":"true",
            "capabilities.add.1":"NET_ADMIN",
            "capabilities.add.2":"SYS_ADMIN",
            "capabilities.drop.1":"ALL"
        },
    )
    chi.container.wait_for_active("5gcn-7-spgwu")
logger.success("5gcn-7-spgwu is up.")

2023-08-06 22:52:30.167 | SUCCESS  | __main__:<cell line: 24>:24 - 5gcn-1-mysql is up.
2023-08-06 22:52:30.497 | SUCCESS  | __main__:<cell line: 38>:38 - 5gcn-2-udr is up.
2023-08-06 22:52:30.796 | SUCCESS  | __main__:<cell line: 52>:52 - 5gcn-3-udm is up.
2023-08-06 22:52:31.090 | SUCCESS  | __main__:<cell line: 66>:66 - 5gcn-4-ausf is up.
2023-08-06 22:52:31.389 | SUCCESS  | __main__:<cell line: 80>:80 - 5gcn-5-amf is up.
2023-08-06 22:52:45.768 | SUCCESS  | __main__:<cell line: 94>:94 - 5gcn-6-smf is up.
2023-08-06 22:52:59.956 | SUCCESS  | __main__:<cell line: 116>:116 - 5gcn-7-spgwu is up.


CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

In [ ]:
# Remove everything and clean up

status = get_container_status("5gcn-7-spgwu")
if status:
    chi.container.destroy_container("5gcn-7-spgwu")
    wait_until_container_removed("5gcn-7-spgwu")

status = get_container_status("5gcn-6-smf")
if status:
    chi.container.destroy_container("5gcn-6-smf")
    wait_until_container_removed("5gcn-6-smf")

status = get_container_status("5gcn-5-amf")
if status:
    chi.container.destroy_container("5gcn-5-amf")
    wait_until_container_removed("5gcn-5-amf")

status = get_container_status("5gcn-4-ausf")
if status:
    chi.container.destroy_container("5gcn-4-ausf")
    wait_until_container_removed("5gcn-4-ausf")

status = get_container_status("5gcn-3-udm")
if status:
    chi.container.destroy_container("5gcn-3-udm")
    wait_until_container_removed("5gcn-3-udm")

status = get_container_status("5gcn-2-udr")
if status:
    chi.container.destroy_container("5gcn-2-udr")
    wait_until_container_removed("5gcn-2-udr")

status = get_container_status("5gcn-1-mysql")
if status:
    chi.container.destroy_container("5gcn-1-mysql")
    wait_until_container_removed("5gcn-1-mysql")

logger.info(f"stopped and removed all containers")

corenet = None
try:
    corenet = chi.network.get_network("5gcn-net")
except Exception as ex:
    logger.info(f"could not find 5gcn-net.")

if corenet:
    chi.network.delete_network(corenet["id"])
    logger.success(f"deleted the 5gcn-net")
